In [10]:
import nltk
import glob
from pathlib import Path
import plistlib
from typing import Union, List, Dict, Tuple, Iterable
import pandas as pd
from sequence_matching import lcss, get_string_with_status, read_plist, read_and_tokenize_file, parse_files_name, get_next_item, save_match_result,create_passage_dict, save_result_csv

In [2]:
# Download punkt and pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/jackwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jackwang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
# Read all *.plist files under Result and create a generator that iter over all files
file_list = glob.glob('./DataFolder/Student_Response/Result/*.plist')
passage_dict = create_passage_dict('./DataFolder/Original_CSV/yrs12_passages.csv')

In [4]:
save_match_result(file_list, save_prefix=Path('./DataFolder/Student_Response/Match/'), passage_dict=passage_dict)

In [11]:
save_result_csv(file_list, save_prefix=Path('./DataFolder/Student_Response/Match/'), passage_dict=passage_dict)